In [2]:
%pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.6 MB/s eta 0:00:00


In [25]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
comment = "Too fast "
def analyze_comment_sentiment(comment):
    score = analyzer.polarity_scores(comment)['compound']
    return score

df['sentiment_score'] = df['comment'].apply(analyze_comment_sentiment)


In [26]:
def check_keywords(comment):
    comment = comment.lower()
    weaknesses = []
    if "fast" in comment:
        weaknesses.append("Too fast")
    if "boring" in comment or "read" in comment:
        weaknesses.append("Unengaging")
    if "example" in comment or "real-life" in comment:
        strengths = ["Uses examples"]
    else:
        strengths = []
    return ", ".join(strengths), ", ".join(weaknesses)

df[['comment_strengths', 'comment_weaknesses']] = df['comment'].apply(lambda x: pd.Series(check_keywords(x)))


In [27]:
def generate_suggestion(row):
    suggestions = []

    if row['q4'] < 3 or 'Too fast' in row['comment_weaknesses']:
        suggestions.append("Try slowing down the explanation.")

    if row['sentiment_score'] < -0.2 or 'Unengaging' in row['comment_weaknesses']:
        suggestions.append("Use storytelling or visuals to keep students engaged.")

    if row['comment_strengths']:
        suggestions.append("Students appreciated your " + row['comment_strengths'].lower() + ".")

    if row['sentiment_score'] > 0.4:
        suggestions.append("Positive response overall. Keep up the good work.")

    return " ".join(suggestions)

df['final_suggestion'] = df.apply(generate_suggestion, axis=1)


In [28]:
new = pd.DataFrame({
    'q1': [1], 'q2': [1], 'q3': [2], 'q4': [1], 'q5': [1],
    'sentiment_score': [-0.2]
})

pred = model.predict(new)
print(le.inverse_transform(pred))

['Add examples and slow down']
